## Decision Tree

In [ ]:
#import necessary packages 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, w_value=None, t_value=None):
        #attributes of decision nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.w_value = w_value
        #attribute of leaf nodes
        self.t_value = t_value

In [ ]:
class Decision_tree:
    #class_or_reg attribute can either be 'calssification' or 'regression'
    #split_type attribute can either be 'gini' or 'entropy'
    def __init__(self, class_or_reg, split_type='gini', max_depth=2, min_sample_split=1):
        
        self.root = None
        self.max_depth = max_depth
        self.opt = class_or_reg
        self.split_type = split_type 
        self.min_sample_split = min_sample_split
        
    def gini_classification(self, y):
        
        labels = np.unique(y)
        gini = 0
        prob_sum = 0
        for l in labels:
            prob = len(y[y == l]) / len(y)
            prob_sum += prob**2
            gini = 1 - prob_sum
        return gini
    
    def entropy_classification(self, y):
        
        labels = np.unique(y)
        entropy = 0
        for l in labels:
            prob = len(y[y == l]) / len(y)
            if prob == 0: # when no element in one label, return 0
                return 0
            else:
                entropy += -prob * np.log2(prob)
        return entropy
    
    def se_regression(self, ytrue, yhat): #for regression use this to split, gini or entropy do not work 

        r = ytrue - yhat 
        return np.sum(r**2)
    
    def best_split(self, x, y):
        
        pass

    def build_tree(self, x, y, curr_depth=0):
        
        pass
    
    def fit(self, x, y):
        
        pass
        
    def print_tree(self, tree=None, indent='  '):
        
        pass
    
    def make_prediction(self, x, tree):
        
        pass
    
    def predict(self, x):
        
        pass